# TASNI: Thermal Anomaly Search for Non-Communicating Intelligence

## Analysis Notebook

This notebook generates publication-quality figures and analysis for the TASNI project.

*Heat where there shouldn't be heat.*

---

In [ ]:
# Imports and setup
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend

from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set publication-quality defaults
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['DejaVu Sans']
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['axes.titlesize'] = 16
matplotlib.rcParams['xtick.labelsize'] = 10
matplotlib.rcParams['ytick.labelsize'] = 10
matplotlib.rcParams['legend.fontsize'] = 10
matplotlib.rcParams['figure.dpi'] = 300

DATA_DIR = Path('/Users/dennispalucki/Documents/TASNI')
FIG_DIR = Path('/Users/dennispalucki/Documents/TASNI/output/figures')
FIG_DIR.mkdir(exist_ok=True)

print(f"Data directory: {DATA_DIR}")
print(f"Figure directory: {FIG_DIR}")

In [ ]:
# Load golden targets
golden = pd.read_csv(DATA_DIR / 'golden_targets.csv')
print(f"Golden targets: {len(golden)}")
print(f"\nColumns: {list(golden.columns)}")
print(f"\nScore statistics:")
score_col = 'prime_score' if 'prime_score' in golden.columns else 'anomaly_score'
print(golden[score_col].describe())

In [ ]:
# Figure 1: Sky distribution of golden targets
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='aitoff')

# Convert RA/Dec to radians
ra_rad = np.radians(golden['ra'].values)
dec_rad = np.radians(golden['dec'].values)

# Aitoff projection expects longitude -pi to pi, latitude -pi/2 to pi/2
scatter = ax.scatter(ra_rad - np.pi, dec_rad, c=golden[score_col], 
           cmap='plasma', s=50, alpha=0.8, edgecolors='none')

ax.set_title('Sky Distribution of TASNI Golden Targets', fontsize=16)
fig.colorbar(scatter, ax=ax, label=score_col, shrink=0.6)

plt.tight_layout()
plt.savefig(FIG_DIR / 'sky_distribution.png', dpi=300, bbox_inches='tight')
plt.savefig(FIG_DIR / 'sky_distribution.pdf', bbox_inches='tight')
print(f"Saved: {FIG_DIR / 'sky_distribution.png'}")
plt.close()

In [ ]:
# Figure 2: WISE color-color diagram
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# W1-W2 vs W2-W3
ax1 = axes[0]
golden['w2_w3'] = golden['w3mpro'] - golden['w2mpro']
ax1.scatter(golden['w1_w2_color'], golden['w2_w3'], 
            c=golden[score_col], cmap='plasma', s=30, alpha=0.7)
ax1.set_xlabel('W1 - W2 (mag)')
ax1.set_ylabel('W2 - W3 (mag)')
ax1.set_title('WISE Color-Color Diagram')

# Add labeled regions
ax1.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax1.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# W1-W2 histogram
ax2 = axes[1]
ax2.hist(golden['w1_w2_color'], bins=30, edgecolor='black', alpha=0.7)
ax2.set_xlabel('W1 - W2 (mag)')
ax2.set_ylabel('Count')
ax2.set_title('W1-W2 Color Distribution')

plt.tight_layout()
plt.savefig(FIG_DIR / 'wise_colors.png', dpi=300, bbox_inches='tight')
plt.savefig(FIG_DIR / 'wise_colors.pdf', bbox_inches='tight')
print(f"Saved: {FIG_DIR / 'wise_colors.png'}")
plt.close()

In [ ]:
# Figure 3: Temperature distribution
if 'T_eff_K' in golden.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Filter to reasonable temperatures
    temps = golden['T_eff_K'].dropna()
    temps = temps[(temps > 100) & (temps < 1000)]
    
    ax.hist(temps, bins=40, edgecolor='black', alpha=0.7, color='steelblue')
    ax.set_xlabel('Effective Temperature (K)')
    ax.set_ylabel('Count')
    ax.set_title('Temperature Distribution of Golden Targets')
    
    # Mark room temperature
    ax.axvline(x=300, color='red', linestyle='--', label='300 K')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig(FIG_DIR / 'temperature_distribution.png', dpi=300)
    plt.savefig(FIG_DIR / 'temperature_distribution.pdf')
    print(f"Saved: {FIG_DIR / 'temperature_distribution.png'}")
    plt.close()
else:
    print("T_eff_K column not found")

In [ ]:
# Figure 4: Score distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

score_col = 'prime_score' if 'prime_score' in golden.columns else 'anomaly_score'

ax1 = axes[0]
ax1.hist(golden[score_col], bins=30, edgecolor='black', alpha=0.7)
ax1.set_xlabel(score_col)
ax1.set_ylabel('Count')
ax1.set_title(f'Distribution of {score_col}')

ax2 = axes[1]
ax2.scatter(golden['w1_w2_color'], golden[score_col], 
            c='steelblue', s=30, alpha=0.7)
ax2.set_xlabel('W1 - W2 (mag)')
ax2.set_ylabel(score_col)
ax2.set_title(f'{score_col} vs Color')

plt.tight_layout()
plt.savefig(FIG_DIR / 'score_distribution.png', dpi=300)
plt.savefig(FIG_DIR / 'score_distribution.pdf')
print(f"Saved: {FIG_DIR / 'score_distribution.png'}")
plt.close()

In [ ]:
# Figure 5: UMAP clustering visualization
if 'umap_x' in golden.columns and 'umap_y' in golden.columns:
    fig, ax = plt.subplots(figsize=(10, 10))
    
    scatter = ax.scatter(golden['umap_x'], golden['umap_y'],
                        c=golden[score_col], cmap='plasma', 
                        s=30, alpha=0.7)
    
    plt.colorbar(scatter, label=score_col)
    ax.set_xlabel('UMAP 1')
    ax.set_ylabel('UMAP 2')
    ax.set_title('UMAP Clustering of Golden Targets')
    
    plt.tight_layout()
    plt.savefig(FIG_DIR / 'umap_clustering.png', dpi=300)
    plt.savefig(FIG_DIR / 'umap_clustering.pdf')
    print(f"Saved: {FIG_DIR / 'umap_clustering.png'}")
    plt.close()
else:
    print("UMAP columns not found")

In [ ]:
# Summary statistics table
print("="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"\nGolden targets: {len(golden)}")

# WISE magnitudes
print("\nWISE magnitudes:")
for band in ['w1mpro', 'w2mpro', 'w3mpro', 'w4mpro']:
    if band in golden.columns:
        print(f"  {band}: {golden[band].mean():.2f} ± {golden[band].std():.2f}")

# Colors
print("\nWISE colors:")
for col in ['w1_w2_color', 'w2_w3', 'w3_w4']:
    if col in golden.columns:
        print(f"  {col}: {golden[col].mean():.2f} ± {golden[col].std():.2f}")

# Score
score_col = 'prime_score' if 'prime_score' in golden.columns else 'anomaly_score'
print(f"\n{score_col}:")
print(f"  Mean: {golden[score_col].mean():.2f}")
print(f"  Max: {golden[score_col].max():.2f}")

# Temperature
if 'T_eff_K' in golden.columns:
    temps = golden['T_eff_K'].dropna()
    print(f"\nTemperature:")
    print(f"  Mean: {temps.mean():.0f} K")
    print(f"  Range: {temps.min():.0f} - {temps.max():.0f} K")

---

## Figures Generated

- `sky_distribution.png/pdf` - Aitoff projection of target positions
- `wise_colors.png/pdf` - WISE color-color diagram and histogram
- `temperature_distribution.png/pdf` - Temperature histogram
- `score_distribution.png/pdf` - Score histogram and color vs score
- `umap_clustering.png/pdf` - UMAP visualization of candidates

---

*The forest was never dark.*